___
# **BME 5710 project report**
## Instructor -- Rizwan Ahmad (ahmad.46@osu.edu)
## BME5710 -- Spring 2024

___
### Provide descriptive answers at `???` locations and insert figures or tables at `?content?`

___
### Write your name below.

Answer: Armon Sekhavat

___
### Write the name of your teammates

Answer: Garrett Herb & Arian Seighali

___
### (1.1) Provide a layout of your CNN (6%)

#### The layout should provide all the necessary details about the CNN architecture including number of channels, size of convolution kernels, activation functions, etc. For inspiration, see examples [here](https://www.geeksforgeeks.org/u-net-architecture-explained/), [here](https://www.researchgate.net/figure/The-architecture-of-Unet_fig2_334287825), [here](https://www.researchgate.net/figure/Modified-U-net-network-architecture_fig2_356216368), and [here](https://open-instruction.com/dl-algorithms/overview-of-residual-neural-network-resnet/).

Image is attached in submission folder

___
### (1.2) List *all* non-trivial features of your CNN and the training process. This may include use of dropout, learning rate scheduling, transfer learning, data augmentation, etc. Don't include items that are already covered in the layout provided above.  (1%)

Data augmantation: 
Translation of 0.1 in both x and y direction
Flip with a probability of 0.5 

Batch normmalization:
BN implemented to all upsampling and downsampling

___
### (2) Answer the following questions about your CNN architecture and training. (5%)

___
### (2.1) Provide at least ten hyperparameters that you *could* optimize in your CNN design and training. (1%)

1) Learning rate
2) Dropout rate (for all 8 convloution layers)
3) Flip rate
4) Number of Epochs
5) Batch Size
6) Weight of SSIM in loss function
7) Weight of L1 in loss function
9) Beta 1 for Adam
10) Beta 2 for Adam
11) Weight decay for Adam

Bounds for translational bounds, Bounds for rotation***

___
### (2.2) Now, list the hyperparameters that you *have* optimized. (1%)

1) Number of epochs 
2) Batch size 
3) Weight of SSIM and L1 in loss function
4) Droupout Rate

rotational bounds, translational bounds**** 


___
### (2.3) Describe your stretegy/approach for optimizing hyperparameters. (1%)

Answer: We started with the orginal U-Net architecture, and form there each team member took turns turns implementing some change to the model, that being a tweak to a particular hyperparameter, or an adjustment to the model design. Tried a high number of epochs, 500, then saw a plateu. Initially increased batch size (6), got worse, then decreased it, and it got better. Founded how much SSIM and L1 norm was contributing to loss function, then added weights to help balance this contribution (didn't actually change weights since didn't make a difference. Tried different dropout settings (.1 and .2), never made it better (worse in some cases), dropout at first and last layer did help reduce difference between test and training loss, but test loss did not improve. Adding rotation made it worse.  

___
### (2.4) What loss function and optimizers did you use?  Express the loss function mathematically. (1%)

$$ L1 = \sum_{i=1}^{n} |y_i - \hat{y}_i| $$

Where:

$n$ is the total number of samples


$\hat{y}_i$ is the predicted value of the i-th sample

$| \cdot |$ denotes the absolute value

$$
\text{SSIM} = \frac{{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}}{{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}}
$$

$\mu_x$ and $\mu_y$: These represent the mean values of the pixel intensities in the original image $(\mu_x)$ and the distorted image $(\mu_y)$, respectively. The mean value is calculated by summing up all the pixel intensities and dividing by the total number of pixels.

$C_1$ and $C_2$: These are constants added to the formula to stabilize the division when the denominator approaches zero. They are typically small positive values.

$\sigma_{xy}$: This represents the covariance between the pixel intensities of the original and distorted images. It measures the linear relationship between the two images.

$\sigma_x$ and $\sigma_y$: These represent the standard deviations of the pixel intensities in the original image $(\sigma_x)$ and the distorted image $(\sigma_y)$, respectively. The standard deviation is a measure of the spread or variability of the pixel intensities.

*Adam was used as the optimizer for this model*

___
### (2.5) Calculate the number of learnable parameters in your final CNN. How does that number compare with the number of training samples? Is your network overfitting or underfitting and how did you arrive at that conclusion? Explain that in the context of loss vs. epoch and NMSE vs. epoch curves for training and validation datasets. (1%)

In [33]:
# Considering batch norm has 2 learnable parameters per channel:
layer1 = (3 * 3 * 1 * 64 + 64) + (3 * 3 * 64 * 64 + 64)
layer2 = (2 * 64) + (3 * 3 * 64 * 128 + 128) + (3 * 3 * 128 * 128 + 128)
layer3 = (2 * 128) + (3 * 3 * 128 * 256 + 256) + (3 * 3 * 256 * 256 + 256)
layer4 = (2 * 256) + (3 * 3 * 256 * 512 + 512) + (3 * 3 * 512 * 512 + 512)
layer5 = (2 * 512) + (3 * 3 * 512 * 1024 + 1024) + (3 * 3 * 1024 * 1024 + 1024)
layer6 = (2 * 2 * 1024 * 512 + 512) + (2 * 512) + (3 * 3 * 1024 * 512 + 512) + (3 * 3 * 512 * 512 + 512)
layer7 = (2 * 2 * 512 * 256 + 256) + (2 * 256) + (3 * 3 * 512 * 256 + 256) + (3 * 3 * 256 * 256 + 256)
layer8 = (2 * 2 * 256 * 128 + 128) + (2 * 128) + (3 * 3 * 256 * 128 + 128) + (3 * 3 * 128 * 128 + 128)
layer9 = (2 * 2 * 128 * 64 + 64) + (2 * 64) + (3 * 3 * 128 * 64 + 64) + (3 * 3 * 64 * 64 + 64) + (1 * 1 * 64 * 1 + 1)
total = layer1 + layer2 + layer3 + layer4 + layer5 + layer6 + layer7 + layer8 + layer9
print("Total # of Learnable Parameters: ", total)

#Overfitting, training samples always perform better

Total # of Learnable Parameters:  31034433


___
### (3.1) Insert (or draw using Markdown) a table that summarizes NMSE and SSIM for (i) noisy images, (ii) images denoised with TrivialNet model included in `starter_code.ipynb`, and (iii) images denoised with your CNN. Include the metrics from training, validation, and test datasets. (6%)

Per our last run:

|             | Noisy vs Clean | TrivialNet vs Clean | DeNet vs Clean |
| ----------- | ----------- | ----------- | ----------- |
| Training NMSE      | -7.935601       | -0.580696       | -19.698793       |
| Training SSIM   | 0.639227        | -0.049679       | 0.962014       |
| Validation NMSE       | -8.009263       | -0.609903       | -19.472027       |
| Validation SSIM       | 0.647057       | -0.057882       | 0.962750       |
| Testing NMSE       | -7.956191       | -0.522039       | -19.660973       |
| Testing SSIM       | 0.674077       | -0.059171       | 0.967844       |

___
### (3.2) When it comes to comparing images, what does SSIM capture that NMSE does not? (1%)

Answer: ???

___
### (4.1) Display a figure where the first row (from left to right) shows an example of clean image, noisy image, image denoised with TrivialNet, and the image denoised with your CNN, and the second row shows corresponding error maps after 3-fold amplification. Select the image from the test dataset. (5%)

Image is attached in submission folder

___
### (4.2) From (4.1), identify which image features are well-preserved by your denoiser and which are lost. Additionally, describe how you could further improve the performance of your denoiser if given more time and resources. (1%)

Adding addtional data, addtional convolutional layers, fine tuning learning rate/trying something other than adam